# Using local Large Language Models

So far, we have used OpenAI's GPT 3.5 models through their API. These models are highly capable, but they offer very little transparency when it comes to the usage and privacy of the data we send through OpenAI's servers.

If we want to take full control over our data, a local Large Language Model may be the solution. Free and open-source Large Language Models are rapidly becoming more available and more capable. While the ones that rival commercial models tend to be huge and need large hardware requirements, there are also many smaller models that, depending on the task, may be perfectly suitable with the right prompts and context.

_LangChain_ supports all kinds of models, including many open-source models. In this notebook, we will show off these capabilitiyes by using a fully local LLM to summarize a document.


## Importing the documents

For this section, we will use the Federalist Papers once again. As in [the previous notebook](02-summarizing.ipynb), we start out by loading the documents.


In [ ]:
from pathlib import Path

docs_dir = Path(
    "~/shared/RR-workshop-data/federalist-papers-dataset/split"
).expanduser()

In [ ]:
from langchain.document_loaders import DirectoryLoader

doc_loader = DirectoryLoader(docs_dir, show_progress=True, recursive=False)
docs = doc_loader.load()

## Loading a model

Since we want to use a locally stored model, we need to acquire a copy of the model itself instead of just calling an API like we did before. Open-source Large Language Model are all over the internet, but one great hub for a curated selection of such models is [GPT4All](https://gpt4all.io/).

For this demonstration, we chose their model from the Falcon family. Just like the dataset, the model file is already provided for you if you are running this on Dartmouth's JupyterHub. Otherwise, you need to [download the model](https://gpt4all.io/models/gguf/gpt4all-falcon-q4_0.gguf) and adjust the path in the following section accordingly.


In [ ]:
from langchain.llms import GPT4All
from pathlib import Path

model_path = Path(
    "~/shared/RR-workshop-data/pretrained-models/llm/gpt4all-falcon-q4_0.gguf"
).expanduser()

llm = GPT4All(model=str(model_path))

## Preparing the text

One of the main limitations when using open-source models is the size of the context window. For this model, the context window is only 2048 tokens. The summarization chain allows for a built-in map-reduce approach to deal with the context window limitation, but the small window size here means that even the very first _map_ step cannot be performed for an entire document at once. We therefore need to split up each document into smaller chunks.

_LangChain_ offers [a number of text splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/#text-splitters), but in this case we will simply chunk the text into bits of 500 non-overlapping characters:


In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
texts = text_splitter.split_documents(docs[:1])

## Summarize using the local model

Now that the document is "digestible" for the LLM, we can utilize the convenience funcition `load_summarize_chain()` to quickly set up and run a _map-reduce_ chain:


In [ ]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm=llm, chain_type="map_reduce")

print(chain.run(texts))

It is certainly much slower than the commercial API, but not bad for such a small model!

Feel free to experiment some more with the local model, and check the slide deck for some next steps!


<table >
<tbody>
  <tr>
    <td style="padding:0px;border-width:0px;vertical-align:center">    
    Created by Simon Stone for Dartmouth College Library under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons CC BY-NC 4.0 License</a>.<br>For questions, comments, or improvements, email <a href="mailto:researchdatahelp@groups.dartmouth.edu">Research Data Services</a>.
    </td>
    <td style="padding:0 0 0 1em;border-width:0px;vertical-align:center"><img alt="Creative Commons License" src="https://i.creativecommons.org/l/by/4.0/88x31.png"/></td>
  </tr>
</tbody>
</table>
